# Génération des réseaux de formes à partir des règles
- Swim1 basé sur l'échantillon initial
 - Gén-1 : génération des formes d'après les contextes phonologiques
 - Gén-2 : génération du réseau d'après les contextes phonologiques
 - Filt-1 : extraction du sous-réseau symétrique
 - Filt-2 : génération du réseau non-orienté correspondant à Filt-1
 - Filt-3 : extraction des cliques maximales & fidèles
- Swim2 basé sur le réseau calculé par Swim1
 - Exp : génération d'un nouvel échantillon basé sur Swim1
 - Gén-1 : génération des formes sans contexte phonologique
 - Gén-2 : génération du réseau sans contexte phonologique
 - Filt-1 : extraction du sous-réseau symétrique
 - Filt-2 : génération du réseau non-orienté correspondant à Filt-1
 - Filt-3 : extraction des cliques maximales & fidèles
- Évaluation  

## Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [1]:
# -*- coding: utf8 -*-
import codecs,operator,datetime,os,glob,cellbell
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pickle
import networkx as nx
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False
from __future__ import print_function

In [2]:
%matplotlib inline

In [3]:
import yaml

In [4]:
from ipywidgets import FloatProgress
from IPython.display import display, HTML

In [5]:
import datetime
def dateheure():
    return datetime.datetime.utcnow().strftime('%y%m%d%H%M')

In [6]:
saut="\n"

### Préparation des matrices de traits

In [7]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')

# Choix de l'échantillon et des règles
- *sampleFile* est le nom de l'échantillon de départ
- *analysisPrefix* est une partie du nom des règles

In [10]:
filePrefix="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/LongitudinalesRnd/Longitudinal"
filePrefix="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/lexique3/Longitudinal-Lexique3"
sampleFiles=glob.glob(filePrefix+"*.pkl")
def prefixEchantillon(numero,sampleType="",casesType=""):
    candidats=[]
#    matchFile=ur"^.*/Longitudinal(-%s-T\d+-F\d+)%s\.pkl"%(numero,sampleType+casesType)
    matchFile=ur"^.*/Longitudinal-Lexique3(-%s-T\d+-F\d+)%s\.pkl"%(numero,sampleType+casesType)
    print (matchFile)
    for sample in sampleFiles:
        m=re.match(matchFile,sample)
        if m:
#            print (sample)
#            print (m.group(1))
            candidats.append(m.group(1))
    if len(candidats)==1:
        return candidats[0]
    else:
        print ("PB pas de nom unique correspondant",len(candidats),numero,sampleType,casesType)

In [11]:
goldPrefix="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/"
goldFile="MGC-171229-Verbes3.pkl"
sampleNumero="00"
sampleType="-X"
casesType="-Morphomes"
#sampleType=""
casesType=""
print (sampleNumero,sampleType,casesType)
if casesType:
    sampleNumber=prefixEchantillon(sampleNumero,sampleType,casesType)
else:
    sampleNumber=prefixEchantillon(sampleNumero)
print (sampleNumber)
paperPrefix="-ISMo"
genFormeVotes=True
genCliques=True
listeFormesOutput=["FS","FP"]
genDigraphe=False
genGraphe=False
plotDistributionCliques=False
if casesType:
    samplePrefix=filePrefix+"%s-%s"%(sampleNumber,sampleType.strip("-"))+casesType
    tirageFile=samplePrefix+".pkl"
elif sampleType:
    samplePrefix=filePrefix+"%s-%s"%(sampleNumber,sampleType.strip("-"))
    tirageFile=filePrefix+sampleNumber+".pkl"
else:
    samplePrefix=filePrefix+"%s-X"%(sampleNumber)
    tirageFile=filePrefix+sampleNumber+".pkl"
sampleFile=samplePrefix+"-paradigmes.csv"
initialFile=filePrefix+prefixEchantillon(sampleNumero)+sampleType+"-paradigmes.csv"
print (initialFile)
analysisPrefix=samplePrefix
if debug: 
    logfile_name=analysisPrefix+"-network.log"
    logfile = codecs.open(logfile_name,mode='w',encoding="utf8")

00 -X 
^.*/Longitudinal-Lexique3(-00-T\d+-F\d+)\.pkl
-00-T150000000-F73539
^.*/Longitudinal-Lexique3(-00-T\d+-F\d+)\.pkl
/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/lexique3/Longitudinal-Lexique3-00-T150000000-F73539-X-paradigmes.csv


In [12]:
#tirage=filePrefix+'-'+sampleNumber+'-Tirage-'+sampleType+'-170503-1907'+casesType+'.pkl'
#tirage=filePrefix+sampleNumber+casesType+'.pkl'
with open(tirageFile, 'rb') as input:
    sampleTirage = pickle.load(input)

In [13]:
if not "morphome" in sampleTirage.columns.tolist():
    sampleTirage["morphome"]=sampleTirage["case"]
morphomeCases=sampleTirage[["case","morphome"]].drop_duplicates().to_dict()
morphomeCases

{'case': {0: u'pi3S',
  1: u'ai3S',
  2: u'ai1S',
  3: u'ii3P',
  4: u'ii3S',
  5: u'pP',
  6: u'pi1S',
  7: u'pI2S',
  9: u'pi3P',
  10: u'inf',
  11: u'fi3S',
  12: u'fi1S',
  13: u'pc3P',
  14: u'pc1S',
  15: u'pc2S',
  16: u'pc3S',
  17: u'pi2S',
  18: u'pI2P',
  19: u'pi2P',
  20: u'pI1P',
  21: u'ai3P',
  22: u'ppMS',
  23: u'ppFS',
  24: u'ppFP',
  25: u'ppMP',
  29: u'ii1S',
  30: u'ii2S',
  33: u'ai2S',
  37: u'ps3S',
  39: u'ps3P',
  47: u'fi2S',
  48: u'fi2P',
  49: u'pc2P',
  50: u'pc1P',
  51: u'fi1P',
  52: u'fi3P',
  54: u'ps2S',
  57: u'ii2P',
  58: u'ii1P',
  60: u'pi1P',
  61: u'ai1P',
  62: u'is3S',
  80: u'ps1S',
  179: u'is3P',
  602: u'is2S',
  740: u'ps2P',
  2708: u'is1S',
  2710: u'is1P',
  2738: u'ps1P',
  2743: u'ai2P',
  17710: u'is2P'},
 'morphome': {0: u'pi3S',
  1: u'ai3S',
  2: u'ai1S',
  3: u'ii3P',
  4: u'ii3S',
  5: u'pP',
  6: u'pi1S',
  7: u'pI2S',
  9: u'pi3P',
  10: u'inf',
  11: u'fi3S',
  12: u'fi1S',
  13: u'pc3P',
  14: u'pc1S',
  15: u'pc2S',

In [14]:
casesMC=morphomeCases["case"]
morphomesMC=morphomeCases["morphome"]
dictMorphomeCases={}
for element in casesMC:
    dictMorphomeCases[casesMC[element]]=morphomesMC[element].split("/")
dictMorphomeCases

{u'ai1P': [u'ai1P'],
 u'ai1S': [u'ai1S'],
 u'ai2P': [u'ai2P'],
 u'ai2S': [u'ai2S'],
 u'ai3P': [u'ai3P'],
 u'ai3S': [u'ai3S'],
 u'fi1P': [u'fi1P'],
 u'fi1S': [u'fi1S'],
 u'fi2P': [u'fi2P'],
 u'fi2S': [u'fi2S'],
 u'fi3P': [u'fi3P'],
 u'fi3S': [u'fi3S'],
 u'ii1P': [u'ii1P'],
 u'ii1S': [u'ii1S'],
 u'ii2P': [u'ii2P'],
 u'ii2S': [u'ii2S'],
 u'ii3P': [u'ii3P'],
 u'ii3S': [u'ii3S'],
 u'inf': [u'inf'],
 u'is1P': [u'is1P'],
 u'is1S': [u'is1S'],
 u'is2P': [u'is2P'],
 u'is2S': [u'is2S'],
 u'is3P': [u'is3P'],
 u'is3S': [u'is3S'],
 u'pI1P': [u'pI1P'],
 u'pI2P': [u'pI2P'],
 u'pI2S': [u'pI2S'],
 u'pP': [u'pP'],
 u'pc1P': [u'pc1P'],
 u'pc1S': [u'pc1S'],
 u'pc2P': [u'pc2P'],
 u'pc2S': [u'pc2S'],
 u'pc3P': [u'pc3P'],
 u'pc3S': [u'pc3S'],
 u'pi1P': [u'pi1P'],
 u'pi1S': [u'pi1S'],
 u'pi2P': [u'pi2P'],
 u'pi2S': [u'pi2S'],
 u'pi3P': [u'pi3P'],
 u'pi3S': [u'pi3S'],
 u'ppFP': [u'ppFP'],
 u'ppFS': [u'ppFS'],
 u'ppMP': [u'ppMP'],
 u'ppMS': [u'ppMS'],
 u'ps1P': [u'ps1P'],
 u'ps1S': [u'ps1S'],
 u'ps2P': [u'ps2P']

# Préparation du calcul des analogies

### Calcul de la différence entre deux formes

In [15]:
def diff(mot1,mot2):
    result=[]
    diff1=""
    diff2=""
    same=""
    vide="."
    lmax=max(len(mot1),len(mot2))
    lmin=min(len(mot1),len(mot2))
    for index in range(lmax):
        if index < lmin:
            if mot1[index]!=mot2[index]:
                diff1+=mot1[index]
                diff2+=mot2[index]
                same+=vide
            else:
                same+=mot1[index]
                diff1+=vide
                diff2+=vide
        elif index < len(mot1):
            diff1+=mot1[index]
        elif index < len(mot2):
            diff2+=mot2[index]
    diff1=diff1.lstrip(".")
    diff2=diff2.lstrip(".")
#    return (same,diff1,diff2,diff1+"_"+diff2)
    return (diff1+"-"+diff2)

### Accumulation des paires appartenant à un patron

In [16]:
def rowDiff(row, patrons):
    result=diff(row[0],row[1])
    if not result in patrons:
        patrons[result]=(formesPatron(),formesPatron())
    patrons[result][0].ajouterFormes(row[0])
    patrons[result][1].ajouterFormes(row[1])
    return (result[0],result[1])

### Transformation d'un patron en RegExp

In [17]:
def patron2regexp(morceaux):
    result="^"
    for morceau in morceaux:
        if morceau=="*":
            result+="(.*)"
        elif len(morceau)>1:
            result+="(["+morceau+"])"
        else:
            result+=morceau
    result+="$"
    result=result.replace(")(","")
    return result

### Substitution de sortie 
???

In [18]:
def remplacementSortie(sortie):
    n=1
    nsortie=""
    for lettre in sortie:
        if lettre==".":
            nsortie+="\g<%d>"%n
            n+=1
        else:
            nsortie+=lettre
    return nsortie

# Classe pour la gestion des patrons, des classes et des transformations

In [19]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            if debug: print ("le numéro de forme n'est pas dans [1,2]",n, file=logfile)

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
        self.classeCF={}
        self.nbClasseCF={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeFormeCF=[]
        regleFormeCF=""
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            filterF1=".*"+patron.split("-")[0]+"$"
            if re.match(filterF1,forme1):
                classeFormeCF.append(patron)
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleFormeCF=patron
            filterF1=self.patrons[patron]
            if re.match(filterF1,forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseFormeCF=", ".join(classeFormeCF)
        if not idClasseFormeCF in self.classeCF:
            self.classeCF[idClasseFormeCF]={}
            self.nbClasseCF[idClasseFormeCF]=0
        if not regleFormeCF in self.classeCF[idClasseFormeCF]:
            self.classeCF[idClasseFormeCF][regleFormeCF]=0
        self.nbClasseCF[idClasseFormeCF]+=1
        self.classeCF[idClasseFormeCF][regleFormeCF]+=1
        
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme,contextFree=True):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if contextFree:
                filterF1=".*"+patron.split("-")[0]+"$"
            else:
                filterF1=self.patrons[patron]
            if re.match(filterF1,forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if contextFree:
                nbClasse=self.nbClasseCF
                classe=self.classeCF
            else:
                nbClasse=self.nbClasse
                classe=self.classe
            if idClasseForme in nbClasse:
                nTotal=nbClasse[idClasseForme]
                for patron in classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(classe[idClasseForme][patron])/nTotal
            else:
                if debug:
                    print (forme, file=logfile)
                    print ("pas de classe",idClasseForme, file=logfile)
                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
        else:
            if debug:
                print (forme, file=logfile) 
                print ("pas de patron", file=logfile)
        return sortieForme
        

## Appliquer la formule de calcul des différences entre chaines à chaque ligne

>si il y a au moins une ligne

>>on applique la différence à la ligne

>>on calcule les deux patrons par suppression des points initiaux

>>on renvoie le groupement par patrons (1&2)

>sinon

>>on renvoie le paradigme vide d'origine

In [20]:
def rapports(paradigme):
    if len(paradigme.columns.values.tolist())==2:
        (case1,lexeme)= paradigme.columns.values.tolist()
        case2=case1
    else:
        (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    classes=paireClasses(case1,case2)
    if len(paradigme)>0:
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
        for regle in regles1:
            classes.ajouterPatron(1,regle,regles1[regle])
        for regle in regles2:
            classes.ajouterPatron(2,regle,regles2[regle])
        paradigme.apply(lambda x: classes.ajouterPaire(x[case1],x[case2]), axis=1)
    (classes1,classes2)=classes.calculerClasses()
    return (classes1,classes2)

### Dédoubler les lignes avec des surabondances dans *colonne*
>identifier une ligne avec surabondance

>>ajouter les lignes correspondant à chaque valeur

>>ajouter le numéro de la ligne initiale dans les lignes à supprimer

>supprimer les lignes avec surabondance

NB : il faut préparer le tableau pour avoir une indexation qui permette l'ajout des valeurs individuelles et la suppression des lignes de surabondances

In [21]:
def splitCellMates(df,colonne):
    '''
    Calcul d'une dataframe sans surabondance par dédoublement des valeurs
    '''
    test=df.reset_index()
    del test["index"]
    splitIndexes=[]
    for index,ligne in test.iterrows():
        if "," in ligne[colonne]:
            valeurs=set(ligne[colonne].split(","))
            nouvelleLigne=ligne
            for valeur in valeurs:
                nouvelleLigne[colonne]=valeur
                test=test.append(nouvelleLigne,ignore_index=True)
            splitIndexes.append(index)
    if splitIndexes:
        test=test.drop(test.index[splitIndexes])
    return test


# Lecture de l'échantillon

In [22]:
phonologicalMap=sampleType.strip("-")
if debug: print(phonologicalMap)
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
if phonologicalMap=="N":
    neutralisations=neutralisationsNORD
elif phonologicalMap=="S":
    neutralisations=neutralisationsSUD
else:
    neutralisations=(u"",u"")
    phonologicalMap=("X")
bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [23]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

### Vérification de la phonotactique des glides du français
- si *prononciation* est *None* renvoyer *None*
- ajout de diérèses dans les séquences mal-formées
- vérification des séquences consonne+glide à la finale

In [24]:
dierese={"j":"ij", "w":"uw","H":"yH","i":"ij","u":"uw","y":"yH"}

In [25]:
def checkFrench(prononciation):
    if prononciation:
        result=recoder(prononciation)
        m=re.match(ur"^.*([^ieèEaOouy926êôâ])[jwH]$",result)
        if m:
            print ("pb avec un glide final", prononciation)
        m=re.match(ur"(.*[ptkbdgfsSvzZ][rl])([jwH])(.*)",result)
        if m:
            n=re.search(ur"[ptkbdgfsSvzZ][rl](wa|Hi|wê)",result)
            if not n:
                glide=m.group(2)
                result=m.group(1)+dierese[glide]+m.group(3)
        m=re.match(ur"(.*)([iuy])([ieEaOouy].*)",result)
        if m:
            glide=m.group(2)
            result=m.group(1)+dierese[glide]+m.group(3)
        result=result.replace("Jj","J")
    else:
        result=prononciation
    return result

In [26]:
paradigmes=pd.read_csv(sampleFile,sep=";",encoding="utf8")
del paradigmes[u"Unnamed: 0"]
paradigmes=paradigmes.dropna(axis=1,how='all')

In [27]:
goldCases=paradigmes.columns.tolist()
goldCases.remove("lexeme")
#goldCases

- sampleCases pour la liste des cases effectivement représentées dans le corpus de départ 

In [28]:
sampleCases=paradigmes.columns.values.tolist()
sampleCases.remove(u"lexeme")
#sampleCases

In [29]:
paradigmes.stack().value_counts(dropna=True).sum()

78823

In [30]:
len(paradigmes.dropna(thresh=1)["lexeme"])

5384

## Préparation des données initiales et de référence

In [31]:
listeTest=paradigmes.dropna(thresh=1)["lexeme"].values.tolist()
#listeTest=[u"asseoir",u"balayer",u"manger"]
nbVerbes=len(listeTest)
#print (nbVerbes)

### Préparation des formes initiales

In [32]:
initialParadigmes=pd.read_csv(initialFile,sep=";",encoding="utf8")
del initialParadigmes[u"Unnamed: 0"]
initialParadigmes=initialParadigmes.dropna(axis=1,how='all')

In [33]:
initialForms=pd.melt(initialParadigmes[paradigmes["lexeme"].isin(listeTest)],id_vars=["lexeme"]).dropna()

initialForms["lexeme-case"]=initialForms["lexeme"]+"-"+initialForms["variable"]
initialForms.drop(labels=["lexeme","variable"],axis=1,inplace=True)

initialForms.set_index(["lexeme-case"],inplace=True)

initialFormsIndex=initialForms.index.tolist()

### Préparation des formes de référence

In [36]:
#with open(goldPrefix+"/"+goldFile,"rb") as input:

# with open(goldPrefix+goldFile,"rb") as input:
#     lexiqueGold=pickle.load(input)
lexiqueGold=pd.read_pickle(path=goldPrefix+goldFile)

'''Rectifications phonologiques'''
lexiqueGold["phono"]=lexiqueGold["phono"].apply(lambda x: checkFrench(x))
completeParadigmes=pd.pivot_table(lexiqueGold, values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()

'''Mise en liste des formes de références'''
goldTestForms=pd.melt(completeParadigmes[completeParadigmes["lexeme"].isin(listeTest)],id_vars=["lexeme"]).dropna()
goldTestForms["lexeme-case"]=goldTestForms["lexeme"]+"-"+goldTestForms["case"]
goldTestForms.drop(labels=["lexeme","case"],axis=1,inplace=True)
goldTestForms.set_index(["lexeme-case"],inplace=True)

'''Extraction des formes de références pertinentes'''
goldForms=goldTestForms.loc[~goldTestForms.index.isin(initialFormsIndex)]
goldFormsIndex=goldForms.index.tolist()

# Lecture des règles

In [37]:
with open(analysisPrefix+'-Regles.pkl', 'rb') as input:
    resultatsLecture = pickle.load(input)

### Comparer les cases analysées avec l'ensemble de toutes les cases

In [38]:
analyseCases=list(set([case for (case,autre) in resultatsLecture.keys()]))
if sorted(analyseCases)!=sorted(goldCases):
    print ("Attention l'analyse ne comprend pas toutes les cases")
    print (sorted(analyseCases))
    print (sorted(goldCases))

# Préparations pour la génération des formes

In [39]:
class paradigmeDistribution:
    '''
    Gestion des distributions dans les cases du paradigme
    '''

    def __init__(self,lexeme):
        self.lexeme=lexeme
        self.formes={i:{} for i in analyseCases}

    def ajouterFormes(self,case,formes,coef=1.0):
        for forme in formes:
            if not forme in self.formes[case]:
                self.formes[case][forme]=0
            self.formes[case][forme]+=formes[forme]*coef
            
    def normaliserDistributions(self,caseListe=analyseCases):
        normalesDistributions={i:{} for i in caseListe}
        for case in caseListe:
            total=0
            for element in self.formes[case]:
                total+=self.formes[case][element]
            for element in self.formes[case]:
                normalesDistributions[case][element]=float(self.formes[case][element])/total
        return normalesDistributions
        

In [40]:
def generateForms(lexeme,contextFree=False):
    candidats=paradigmeDistribution(lexeme)
    casesSamples=paradigmes[paradigmes["lexeme"]==lexeme].columns[paradigmes[paradigmes["lexeme"]==lexeme].notnull().iloc[0]].tolist()
    casesSamples.remove("lexeme")
    for caseDepart in casesSamples:
        formeDepart=paradigmes[paradigmes["lexeme"]==lexeme][caseDepart].iloc[0]
        if debug: print (caseDepart,formeDepart, file=logfile)
#        if formeDepart!="nan":
        for case in analyseCases:
            if debug: print (case, file=logfile)
            if not isinstance(resultatsLecture[(caseDepart, case)],str):
                if "," in formeDepart:
                    formesDepart=formeDepart.split(",")
                    coef=1.0/len(formesDepart)
                    for element in formesDepart:
                        candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(element,contextFree),coef)
                else:
                    candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(formeDepart,contextFree))
            else: 
                if debug: print ("str", resultatsLecture[(caseDepart, case)], file=logfile)
    return candidats

In [41]:
def ajouterPoint(lexeme,forme,case,digraphe,graphe):
    pointName="%s-%s-%s"%(lexeme,forme,case)
#    if not pointName in digraphe.nodes():
    tam=case[:2]
    if tam=="in": tam="inf"
    digraphe.add_node(pointName, tam='"%s"'%tam)
    graphe.add_node(pointName, tam='"%s"'%tam)
    return pointName

def ajouterFleche(pointDepart,pointSortie,coef,digraphe,graphe):
    digraphe.add_edge(pointDepart,pointSortie,weight=float(coef))
    if digraphe.has_edge(pointSortie,pointDepart):
        coefGraphe=float(digraphe.edge[pointSortie][pointDepart]["weight"]+coef)/2
        graphe.add_edge(pointDepart,pointSortie,weight=coefGraphe)

In [42]:
def generateParadigms(generation1,genDigraphe=True,contextFree=False):
    lexeme=generation1.lexeme
    distributionInitiale=generation1.normaliserDistributions()
    candidats=paradigmeDistribution(lexeme)
    digraphe=nx.DiGraph()
    graphe=nx.Graph()    
    for caseDepart in analyseCases:
        for formeDepart in distributionInitiale[caseDepart]:
            if formeDepart:
                pointDepart=ajouterPoint(lexeme,formeDepart,caseDepart,digraphe,graphe)
                coefDepart=distributionInitiale[caseDepart][formeDepart]
                if debug: print (caseDepart,formeDepart, file=logfile)
                for caseSortie in analyseCases:
                    distributionSortieBrute=resultatsLecture[(caseDepart, caseSortie)].sortirForme(formeDepart,contextFree)
                    if distributionSortieBrute:
                        if not genDigraphe:
#                            print ("brute",distributionSortieBrute)
                            distributionSortie={f:distributionSortieBrute[f] for f in distributionSortieBrute if f in distributionInitiale[caseSortie]}
                        else:
                            distributionSortie=distributionSortieBrute
#                        print ("filtre",distributionSortie)
#                        print (distributionInitiale[caseSortie])
                        if debug: print (caseSortie,distributionSortie,distributionInitiale[caseDepart], file=logfile)
                        candidats.ajouterFormes(caseSortie,distributionSortie,distributionInitiale[caseDepart][formeDepart])
                        for formeSortie in distributionSortie:
                            pointSortie=ajouterPoint(lexeme,formeSortie,caseSortie,digraphe,graphe)
                            coefSortie=distributionSortie[formeSortie]
                            ajouterFleche(pointDepart,pointSortie,float(coefDepart*coefSortie),digraphe,graphe)
    return (candidats,digraphe,graphe)

# Génération d'un jeu de cliques

In [43]:
def generate(lexeme,genDigraphe=True,contextFree=False):
#    print (lexeme,end=", ")
    generation1=generateForms(lexeme,contextFree)
#    print ("génération 2",end=", ")
    (generation2,lexDigraphe,lexGraphe)=generateParadigms(generation1,genDigraphe,contextFree)
    lexCliques=list(nx.algorithms.clique.find_cliques(lexGraphe))
#    print (lexCliques)
#    print ("génération 3")
    return (generation2,lexDigraphe,lexGraphe,lexCliques)

In [44]:
paradigmes.dropna(thresh=1).count().sum()-paradigmes.dropna(thresh=1)["lexeme"].count()

73439

#### Calculer le score de la clique

In [45]:
def cliqueScore(clique,graph):
    score=0
    if len(clique)>1:
        for (depart,arrivee) in it.combinations_with_replacement(clique,2):
            score+=graph[depart][arrivee]["weight"]
    return score

In [46]:
def splitArrivee(arrivee):
    arriveeMorceaux=arrivee.split("-")
    if len(arriveeMorceaux)<3:
        print (arrivee,arriveeMorceaux)
    lexeme="-".join(arriveeMorceaux[:-2])
    formeArrivee=arriveeMorceaux[-2]
    caseArrivee=arriveeMorceaux[-1]
    return (lexeme,formeArrivee,caseArrivee)
    
# trouver tous les liens vers FS-* et FP-*
# regrouper par forme 
# calculer les proportions
# renvoyer les proportions par forme
# avec le nombre de forme à l'appui
def formeScore(forme,graph):
    scores={}
    scoresNormes={}
    for depart in graph.edge:
        for arrivee in graph.edge[depart]:
            (lexeme, formeArrivee, caseArrivee)=splitArrivee(arrivee)
            if caseArrivee==forme:
#                print (depart, formeArrivee, graph.edge[depart][arrivee])
                if not formeArrivee in scores:
                    scores[formeArrivee]=0
                scores[formeArrivee]+=graph.edge[depart][arrivee]["weight"]
    totalArrivee=0
    for formeArrivee in scores:
        totalArrivee+=scores[formeArrivee]
    for formeArrivee in scores:
        scoresNormes[formeArrivee]=scores[formeArrivee]/totalArrivee
    return (scores,scoresNormes)
        

## Préparations pour SWIM

In [47]:
def generateAnalysis(globDigraphe,globGraphe,contextFree=False):
    numClique=0
    progressBar = FloatProgress(min=0, max=nbVerbes-1,description="Generation (%d verbs)"%nbVerbes)
    display(progressBar)
    for i,element in enumerate(listeTest):
        cliquesScores[element]={}
        cliquesListes[element]={}
    #    if (i%100)==0: print (i, dateheure()[-4:], int(100*float(i)/nbVerbes), end=", ")
        progressBar.value=i
        #print (element)
        result=generate(element,genDigraphe,contextFree)
        (generation,lexDigraphe,lexGraphe,lexCliques)= result
    #    print (generation,lexDigraphe,lexGraphe,lexCliques)
        if genFormeVotes:
            formesScores[element]={}
            formesScoresNormes[element]={}
            for formeOutput in listeFormesOutput:
                (formesScores[element][formeOutput],formesScoresNormes[element][formeOutput])=formeScore(formeOutput,lexDigraphe)
        if genDigraphe:
            globDigraphe=nx.union(globDigraphe,lexDigraphe)
        if genGraphe:
            globGraphe=nx.union(globGraphe,lexGraphe)
        cliques.extend(lexCliques)
        for clique in lexCliques:
            cliquesScores[element][numClique]=cliqueScore(clique,lexGraphe)
            cliquesListes[element][numClique]=clique
            numClique+=1
    return globDigraphe,globGraphe,numClique

In [48]:
def dictCliqueForms(clique):
    result={}
    for element in clique:
        lexeme,forme,case=splitArrivee(element)
        for c in dictMorphomeCases[case]:
            result[c]=forme
    return result

def dictPdRowForms(row):
    result={}
    for case in sampleCases:
        print (case,row[case].values[0])
    return result

def tableZero(case):
    if case in sampleCases:
        return u"Ø"
    else:
        return u"="

def makeTable(dictForms,title=""):
    tabular=[]
    labelTenseCode={"pi":"Present","ii":"Imperfective","ai":"Simple Past","fi":"Future",
                    "ps":"Subjunctive Pres.","is":"Subjunctive Imp.","pc":"Conditional","pI":"Imperative",
                    "inf":"Infinitive",
                    "ppMS":"Past Part. MS","ppMP":"Past Part. MP",
                    "ppFS":"Past Part. FS","ppMP":"Past Part. FP"
                   }
    def makeLine6(tenseCode):
        line=[]
        line.append(r"<th>%s</th>"%labelTenseCode[tenseCode])
        for person in [per+nb for nb in ["S","P"] for per in ["1","2","3"]]:
            case=tenseCode+person
            if (case in dictForms) and (not (type(dictForms[case]) == float and np.isnan(dictForms[case]))):
                line.append(r"<td>%s</td>"%(dictForms[case]))
            else:
                line.append(r"<td>%s</td>"%(tableZero(case)))
        return r"<tr>"+r"".join(line)+r"</tr>"

    def makeLine3(tenseCode):
        line=[]
        line.append(r"<th>%s</th>"%labelTenseCode[tenseCode])
        for person in [per+nb for nb in ["S","P"] for per in ["1","2","3"]]:
            if person in ["2S","1P","2P"]:
                case=tenseCode+person
                if case in dictForms and (not (type(dictForms[case]) == float and np.isnan(dictForms[case]))):
                    line.append(r"<td>%s</td>"%(dictForms[case]))
                else:
                    line.append(r"<td>%s</td>"%(tableZero(case)))
            else:
                line.append(r"<td>%s</td>"%(u"---"))
        return r"<tr>"+r"".join(line)+r"</tr>"
    
    def makeLineNF():
        line=[]
        line.append(r"<th>%s</th>"%"NF")
        for case in ["inf","pP","ppMS","ppMP","ppFS","ppFP"]:
            if case in dictForms and (not (type(dictForms[case]) == float and np.isnan(dictForms[case]))):
                line.append(r"<td>%s</td>"%(dictForms[case]))
            else:
                line.append(r"<td>%s</td>"%(tableZero(case)))
        return r"<tr>"+r"".join(line)+r"</tr>"
    
        
    top=[
        r"<table>",
        r"<caption style='caption-side:bottom;text-align:center'>",
        "Verbe : %s"%title,
        r"</caption>",
#        r"<tr><th/><th>1S</th><th>2S</th><th>3S</th><th>1P</th><th>2P</th><th>3P</th></tr>"
        r"<tr><th/><th>1SG</th><th>2SG</th><th>3SG</th><th>1PL</th><th>2PL</th><th>3PL</th></tr>"
        ]
    bottom=[
        r"</table>"
        ]
    tabular.append("\n".join(top))
    for tenseCode in ["pi","ii","fi","pc", "ps","ai", "is"]:
        tabular.append(makeLine6(tenseCode))
    tabular.append(makeLine3("pI"))
    tabular.append(makeLineNF())
    tabular.append("\n".join(bottom))
    return "\n".join(tabular)    

def diffParadigme(lexeme):
    outLen=lexemeMaxCliques[lexeme][0]
    inLen=paradigmes[paradigmes["lexeme"]==lexeme].notnull().sum(axis=1).values[0]-1
    if outLen>inLen:
        print (lexemeMaxCliques[lexeme][1])
        print (paradigmes[paradigmes["lexeme"]=="grandir"].values)
    return outLen-inLen
    

In [49]:
def checkFidelite(fidelite,clique):
    lFidele=False
    for element in clique:
        if fidelite in element:
            lFidele=True
    return lFidele

def generateCliques(contextFree=False):
    
    def bruteCliques(lexeme,maxCliqueSize=51):
        cliquesBrutes={n+1:0 for n in range(maxCliqueSize)}
        for l in cliquesListes[lexeme].values():
            longueur=len(l)
            if longueur>1:
                if not longueur in cliquesBrutes:
                    cliquesBrutes[longueur]=0
                cliquesBrutes[longueur]+=1
        return cliquesBrutes
    
    globDigraphe=nx.DiGraph()
    globGraphe=nx.Graph()

    globDigraphe,globGraphe,numClique=generateAnalysis(globDigraphe,globGraphe,contextFree)
    print 

    lexemeMaxCliques={}
    lexemeParadigmes={}
    progressBarCliques = FloatProgress(min=0, max=len(cliquesListes)-1,description="Analysis (%d verbes)"%len(cliquesListes))
    display(progressBarCliques)
    for lexeme in cliquesListes:
        progressBarCliques.value+=1
        maxLen=max([len(c) for c in cliquesListes[lexeme].values()])
        lexemeMaxCliques[lexeme]=bruteCliques(lexeme,maxLen)
        print (lexeme,"Nombre de cliques",sum([v for k,v in lexemeMaxCliques[lexeme].iteritems()]))
        maxNbCliques=max([v for k,v in lexemeMaxCliques[lexeme].iteritems()])
        if plotDistributionCliques:
            ax=pd.DataFrame.from_dict(lexemeMaxCliques[lexeme],orient="index").plot(kind="bar",legend=False,grid=True,figsize=(10,3))
            ax.set(xlim=(0,maxLen+.5),ylim=(0,maxNbCliques+10))
            ax.set_xlabel("Clique Size in Cells",fontsize=16)
            ax.set_ylabel("Number of Cliques",fontsize=16)

        dictParadigmes=paradigmes.set_index("lexeme").to_dict(orient="index")

        cliquesFideles={}
        fidelites=[v+"-"+k for k,v in dictParadigmes[lexeme].iteritems() if isinstance(v,unicode)]
        for l in cliquesListes[lexeme].values():
            longueur=len(l)
            if longueur>1:
                fidele=True
                for fidelite in fidelites:
                    if "," in fidelite:
                        fideliteForme,fideliteCase=fidelite.split("-")
                        fideliteFormes=fideliteForme.split(",")
                        fideliteItems=[fideliteF+"-"+fideliteCase for fideliteF in fideliteFormes]
#                        print (fideliteItems)
                        lFidele=True
                        for f in fideliteItems:
                            lFidele=lFidele & checkFidelite(f,l)
                    else:
                        lFidele=checkFidelite(fidelite,l)
                    if not lFidele:
                        fidele=False
                        break
                if fidele:
                    if not longueur in cliquesFideles:
                        cliquesFideles[longueur]=[]
                    cliquesFideles[longueur].append(l)
#                else:
#                    if lexeme==u"bégayer": print ()
#        print ([(k,len(v)) for k,v in cliquesFideles.iteritems()])
        if cliquesFideles:
            maxCliquesCard=max([k for k,v in cliquesFideles.iteritems()])
    #        print (maxCliquesCard)
    #        print (cliquesScores[lexeme])
    #        print (cliquesListes[lexeme])
            lexemeParadigmes[lexeme]=[]
            maxScoreCliques=max([clique for cliqueNumber, clique in cliquesScores[lexeme].items()])
            maxCardScoreNums=[numC for numC, c in cliquesListes[lexeme].items() if c in cliquesFideles[maxCliquesCard]]
            maxCardScore=max([scoreC for numC, scoreC in cliquesScores[lexeme].items() if numC in maxCardScoreNums])
    #        print ("max score among all cliques:",maxScoreCliques)
            print ("max score among faithfull cliques of %d forms:"%maxCliquesCard,maxCardScore)
            for c in cliquesFideles[maxCliquesCard]:
                cNumber=[cliqueNumber for cliqueNumber, clique in cliquesListes[lexeme].items() if clique == c]
                if len(cNumber)!=1:
                    print ("TOO MANY SCORES PROBLEM WITH CLIQUE", cNumber)
    #            print ("Liste n°",cNumber[0],cliquesScores[lexeme][cNumber[0]])
    #            print (sorted(cliquesScores[lexeme].items(), key=operator.itemgetter(1)))
    #            display(HTML(makeTable(dictCliqueForms(c),title=c[0].split("-")[0])))
    #            print (cliquesScores[lexeme][cNumber[0]], maxCardScore)
                if cliquesScores[lexeme][cNumber[0]]==maxCardScore:
                    lexemeParadigmes[lexeme].append(c)
        else:
            lexemeParadigmes[lexeme]=[[lexeme+"-"+f for f in fidelites]]
            print (u"no new faithfull clique, the previous one contained %d forms"%len(lexemeParadigmes[lexeme][0]))
    return lexemeParadigmes

In [50]:
def cutNodeName(nodeName):
    items=nodeName.split("-")
    nbItems=len(items)
    if nbItems>3:
        items=["-".join(items[0:nbItems-2]),items[-2],items[-1]]
    return items

def filledOutClique(cliques):
    if cliques:
        result=[]
        for clique in cliques:
#            print (clique)
            fullClique=[]
            for element in clique:
                if debug: print ("element",element)
                lexeme,forme,case=splitArrivee(element)
                for c in dictMorphomeCases[case]:
                    fullClique.append("-".join([lexeme,forme,c]))
            result.append(fullClique)
        return result
    else:
        return cliques

In [51]:
def extendParadigmes(contextParadigmes,extendMorphomes=False):
    lexemesParadigmeListe=[]
    for lexeme in contextParadigmes:
        if extendMorphomes:
            lexParadigmes=filledOutClique(contextParadigmes[lexeme])
        else:
            lexParadigmes=contextParadigmes[lexeme]
        if len(lexParadigmes)!=1:
            if debug:
                print ("LEXEME WITH A NON UNIQUE PARADIGM PB",len(lexParadigmes),lexeme)
                print (lexParadigmes)
        lexParadigme=lexParadigmes[0]
        for lexForme in lexParadigme:
            lexemesParadigmeListe.append(cutNodeName(lexForme))
    newForms=pd.DataFrame(lexemesParadigmeListe)
    newForms.columns=["lexeme","form","case"]
#    newParadigmes=newForms.pivot(index="lexeme", columns="case", values="form")
    newParadigmes=pd.pivot_table(newForms, values='form', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()
    for i in newParadigmes.itertuples():
#        print (i[0],i[1])
        lexeme=i[1]
        lexemeIndexes=paradigmes.lexeme[paradigmes.lexeme==lexeme].index.tolist()
        if lexemeIndexes:
            lexemeIndex=lexemeIndexes[0]
        else:
            print (i,lexeme,lexemeIndexes)
        newParadigmes.loc[newParadigmes.lexeme==lexeme,"index"]=int(lexemeIndex)
    newParadigmes.set_index("index",inplace=True)   
    return paradigmes.combine_first(newParadigmes)

In [62]:
def countSplits(dfForms):
    dfForms.loc[:,"split"]=dfForms.loc[:,"value"].str.split(",")
    return dfForms["split"].str.len().sum()

def calculerResultats(contextParadigmes,extension="-Swim2"):
    
    '''Préparer le paradigme des prédictions'''
    brutParadigmes=extendParadigmes(contextParadigmes,extendMorphomes=False)
    finalParadigmes=extendParadigmes(contextParadigmes,extendMorphomes=True)
    finalParadigmes.to_csv(path_or_buf=analysisPrefix+"-paradigmes%s.csv"%extension,encoding="utf8",sep=";")
    finalTestForms=pd.melt(finalParadigmes[finalParadigmes["lexeme"].isin(listeTest)],id_vars=["lexeme"]).dropna()
#     finalTestForms["lexeme-case"]=finalTestForms["lexeme"]+"-"+finalTestForms["variable"]
#     finalTestForms.drop(labels=["lexeme","variable"],axis=1,inplace=True)
    finalTestForms["lexeme-case"]=finalTestForms["lexeme"]+"-"+finalTestForms["case"]
    finalTestForms.drop(labels=["lexeme","case"],axis=1,inplace=True)
    finalTestForms.set_index(["lexeme-case"],inplace=True)
    
    '''Soustraire les formes initiales'''
    finalForms=finalTestForms.loc[~finalTestForms.index.isin(initialFormsIndex)]
    finalFormsIndex=finalForms.index.tolist()
    
    '''Calculer les sur/sous-générations'''
    underGeneration=goldForms.loc[~goldForms.index.isin(finalFormsIndex)]
    overGeneration=finalForms.loc[~finalForms.index.isin(goldFormsIndex)]
    
    '''Réduire les prédictions et la référence aux cases communes'''
    predictedForms=finalForms.loc[finalForms.index.isin(goldFormsIndex)]
    actualForms=goldForms.loc[goldForms.index.isin(finalFormsIndex)]
    
    '''Créer un tableau pour les comparaisons'''
    compareForms=predictedForms.copy()
    compareForms.loc[:,"right"]=actualForms.loc[:,"value"]
    
    '''Séparer les cases identiques des cases différentes'''
    sameForms=compareForms[compareForms["value"]==compareForms["right"]]
    diffForms=compareForms[compareForms["value"]!=compareForms["right"]]

    
    '''Sauvegarder les comparatifs'''
    overGeneration.to_csv(path_or_buf=analysisPrefix+"-overGeneration%s.csv"%extension,encoding="utf8")
    underGeneration.to_csv(path_or_buf=analysisPrefix+"-underGeneration%s.csv"%extension,encoding="utf8")
    sameForms.to_csv(path_or_buf=analysisPrefix+"-sameForms%s.csv"%extension,encoding="utf8")
    diffForms.to_csv(path_or_buf=analysisPrefix+"-diffForms%s.csv"%extension,encoding="utf8")
    
    
    '''Transformer les surabondances en liste'''
    diffForms.loc[:,"split-value"]=diffForms.loc[:,"value"].str.split(",")
    diffForms.loc[:,"split-right"]=diffForms.loc[:,"right"].str.split(",")

    '''Transformer les surabondances en set()'''
    diffForms.loc[:,"split-value"]=diffForms.loc[:,"split-value"].apply(set)
    diffForms.loc[:,"split-right"]=diffForms.loc[:,"split-right"].apply(set)
    
    '''Calculer le nombre de formes (y compris surabondances)'''
    nbValues=diffForms["split-value"].str.len().sum()
    nbRights=diffForms["split-right"].str.len().sum()

    '''Calculer les identités et les inclusions'''
    nbIdenticalSets=diffForms[diffForms["split-value"]==diffForms["split-right"]]["split-value"].str.len().sum()
    nbIncludedSets=diffForms[diffForms["split-value"]<diffForms["split-right"]]["split-value"].str.len().sum()
    nbWrongForms=(nbValues-nbIdenticalSets-nbIncludedSets)
    underBonus=(nbRights-nbIdenticalSets-nbIncludedSets)

    UG=countSplits(underGeneration)+underBonus
    OG=countSplits(overGeneration)
    TP=countSplits(sameForms)+nbIdenticalSets+nbIncludedSets
    FP=nbWrongForms
    resultCharacteristics=(UG,OG,TP,FP)
    recall=float(TP)/(UG+TP+FP)
    precision=float(TP)/(OG+TP+FP)
    fMeasure=2*recall*precision/(recall+precision)
    resultMeasures=(precision,recall,fMeasure)
    print ("UG",UG ,"OG",OG,"TP",TP,"FP",FP)
    print ("recall", recall, "precision", precision)
    print (fMeasure)
    return (brutParadigmes,finalParadigmes,resultCharacteristics,resultMeasures)


# SWIM1

In [53]:
cliques=[]
cliquesScores={}
cliquesListes={}

formesScores={}
formesScoresNormes={}

print (datetime.datetime.now())
%time swim1ContextParadigmes=generateCliques()

2019-09-16 12:11:13.041115


escamoter Nombre de cliques 33
max score among faithfull cliques of 44 forms: 896.478423622
anémier Nombre de cliques 104
max score among faithfull cliques of 27 forms: 263.005339887
enquérir Nombre de cliques 344
max score among faithfull cliques of 18 forms: 131.483481634
réfugier Nombre de cliques 247
max score among faithfull cliques of 26 forms: 246.392989928
assembler Nombre de cliques 28
max score among faithfull cliques of 41 forms: 836.28662991
ensanglanter Nombre de cliques 26
max score among faithfull cliques of 43 forms: 918.111963455
projeter Nombre de cliques 287
max score among faithfull cliques of 42 forms: 550.168744395
conceptualiser Nombre de cliques 13
max score among faithfull cliques of 44 forms: 909.007775649
forger Nombre de cliques 41
max score among faithfull cliques of 44 forms: 950.857831671
féminiser Nombre de cliques 17
max score among faithfull cliques of 45 forms: 969.923065502
aider Nombre de cliques 31
max score among faithfull cliques of 46 forms: 104

max score among faithfull cliques of 42 forms: 740.902749691
agrandir Nombre de cliques 627
max score among faithfull cliques of 37 forms: 383.014616574
réveiller Nombre de cliques 158
max score among faithfull cliques of 41 forms: 527.087135162
catalyser Nombre de cliques 13
max score among faithfull cliques of 44 forms: 906.133692401
relativiser Nombre de cliques 185
max score among faithfull cliques of 40 forms: 663.061264685
ranimer Nombre de cliques 18
max score among faithfull cliques of 45 forms: 1012.21779402
moucharder Nombre de cliques 35
max score among faithfull cliques of 44 forms: 906.846590172
incarner Nombre de cliques 46
max score among faithfull cliques of 45 forms: 949.729879799
financer Nombre de cliques 29
max score among faithfull cliques of 43 forms: 853.060805055
convulsionner Nombre de cliques 10
max score among faithfull cliques of 46 forms: 1042.45511719
incuber Nombre de cliques 9
max score among faithfull cliques of 46 forms: 1024.22165123
remplacer Nombre 

max score among faithfull cliques of 44 forms: 870.87049601
insinuer Nombre de cliques 28
max score among faithfull cliques of 38 forms: 705.551118849
réopérer Nombre de cliques 28
max score among faithfull cliques of 42 forms: 792.135422478
formaliser Nombre de cliques 33
max score among faithfull cliques of 44 forms: 854.43500525
exploser Nombre de cliques 105
max score among faithfull cliques of 30 forms: 359.941466431
parafer Nombre de cliques 27
max score among faithfull cliques of 41 forms: 790.527609077
réactualiser Nombre de cliques 13
max score among faithfull cliques of 44 forms: 909.007775649
officialiser Nombre de cliques 25
max score among faithfull cliques of 44 forms: 858.945757927
rugir Nombre de cliques 297
max score among faithfull cliques of 38 forms: 541.172266525
tripatouiller Nombre de cliques 31
max score among faithfull cliques of 39 forms: 720.215012125
écailler Nombre de cliques 85
max score among faithfull cliques of 39 forms: 536.807681069
menacer Nombre de 

max score among faithfull cliques of 41 forms: 783.114613541
destiner Nombre de cliques 37
max score among faithfull cliques of 46 forms: 1036.11503799
mépriser Nombre de cliques 55
max score among faithfull cliques of 46 forms: 950.272439473
mouvoir Nombre de cliques 93
max score among faithfull cliques of 13 forms: 79.9542743011
appendre Nombre de cliques 96
max score among faithfull cliques of 44 forms: 616.818658924
rassurer Nombre de cliques 35
max score among faithfull cliques of 45 forms: 965.842050522
mourir Nombre de cliques 151
max score among faithfull cliques of 43 forms: 726.12397102
suggérer Nombre de cliques 85
max score among faithfull cliques of 42 forms: 783.118585545
prévenir Nombre de cliques 240
max score among faithfull cliques of 44 forms: 872.376370537
nipper Nombre de cliques 19
max score among faithfull cliques of 45 forms: 986.654327167
attitrer Nombre de cliques 12
max score among faithfull cliques of 40 forms: 778.125154404
chiner Nombre de cliques 39
max s

...

max score among faithfull cliques of 37 forms: 566.681024231
inséminer Nombre de cliques 8
max score among faithfull cliques of 45 forms: 1012.95428905
commotionner Nombre de cliques 15
max score among faithfull cliques of 46 forms: 1013.56109597
baratiner Nombre de cliques 35
max score among faithfull cliques of 46 forms: 1036.73655404
refouiller Nombre de cliques 11
max score among faithfull cliques of 39 forms: 729.427973336
sauter Nombre de cliques 75
max score among faithfull cliques of 38 forms: 582.851653723
défroisser Nombre de cliques 152
max score among faithfull cliques of 45 forms: 907.862237363
baver Nombre de cliques 173
max score among faithfull cliques of 36 forms: 558.480636869
architecturer Nombre de cliques 23
max score among faithfull cliques of 45 forms: 959.713587348
ressemer Nombre de cliques 42
max score among faithfull cliques of 38 forms: 575.056139274
totaliser Nombre de cliques 27
max score among faithfull cliques of 44 forms: 837.537514936
dépecer Nombre de

max score among faithfull cliques of 46 forms: 1043.31478972
gaufrer Nombre de cliques 58
max score among faithfull cliques of 6 forms: 20.9357477249
déchoir Nombre de cliques 58
max score among faithfull cliques of 9 forms: 31.8335787855
ravager Nombre de cliques 39
max score among faithfull cliques of 45 forms: 987.414862826
persiller Nombre de cliques 277
max score among faithfull cliques of 39 forms: 486.407371952
accaparer Nombre de cliques 41
max score among faithfull cliques of 42 forms: 778.469992412
offusquer Nombre de cliques 18
max score among faithfull cliques of 42 forms: 887.996486912
râper Nombre de cliques 31
max score among faithfull cliques of 46 forms: 976.84134686
assaillir Nombre de cliques 274
max score among faithfull cliques of 18 forms: 73.6932511758
chevaler Nombre de cliques 9
max score among faithfull cliques of 42 forms: 883.906126559
siéger Nombre de cliques 38
max score among faithfull cliques of 45 forms: 981.764863184
oxyder Nombre de cliques 8
max scor

strier Nombre de cliques 196
max score among faithfull cliques of 34 forms: 301.782594143
décélérer Nombre de cliques 8
max score among faithfull cliques of 44 forms: 951.282760182
goudronner Nombre de cliques 23
max score among faithfull cliques of 46 forms: 1016.48380563
stériliser Nombre de cliques 29
max score among faithfull cliques of 44 forms: 851.458693514
convoiter Nombre de cliques 36
max score among faithfull cliques of 45 forms: 921.468496797
trébucher Nombre de cliques 33
max score among faithfull cliques of 45 forms: 992.127555786
emménager Nombre de cliques 32
max score among faithfull cliques of 44 forms: 953.641209046
pérorer Nombre de cliques 53
max score among faithfull cliques of 43 forms: 800.676206178
tonsurer Nombre de cliques 9
max score among faithfull cliques of 45 forms: 968.006469212
analyser Nombre de cliques 48
max score among faithfull cliques of 44 forms: 830.537164232
émaner Nombre de cliques 17
max score among faithfull cliques of 45 forms: 977.7506874

max score among faithfull cliques of 42 forms: 750.51937688
désoccuper Nombre de cliques 8
max score among faithfull cliques of 46 forms: 1032.42889311
remprunter Nombre de cliques 6
max score among faithfull cliques of 43 forms: 908.568803709
suffire Nombre de cliques 700
max score among faithfull cliques of 24 forms: 114.0171639
endiguer Nombre de cliques 25
max score among faithfull cliques of 45 forms: 1002.72746154
accourcir Nombre de cliques 125
max score among faithfull cliques of 37 forms: 531.477825333
répercuter Nombre de cliques 34
max score among faithfull cliques of 46 forms: 1012.19301377
précéder Nombre de cliques 167
max score among faithfull cliques of 45 forms: 776.723272605
défouler Nombre de cliques 31
max score among faithfull cliques of 43 forms: 912.330710921
exhausser Nombre de cliques 50
max score among faithfull cliques of 38 forms: 595.694767668
baiser Nombre de cliques 286
max score among faithfull cliques of 43 forms: 636.22485187
glousser Nombre de cliques

époustoufler Nombre de cliques 15
max score among faithfull cliques of 40 forms: 799.253849103
remorquer Nombre de cliques 37
max score among faithfull cliques of 44 forms: 948.322450703
pérenniser Nombre de cliques 6
max score among faithfull cliques of 45 forms: 999.147171255
décrire Nombre de cliques 683
max score among faithfull cliques of 35 forms: 329.838075204
démettre Nombre de cliques 180
max score among faithfull cliques of 43 forms: 569.829206156
volatiliser Nombre de cliques 31
max score among faithfull cliques of 44 forms: 852.797977346
entailler Nombre de cliques 103
max score among faithfull cliques of 39 forms: 555.928503557
monter Nombre de cliques 36
max score among faithfull cliques of 43 forms: 885.40616868
réinviter Nombre de cliques 12
max score among faithfull cliques of 45 forms: 969.320041181
seringuer Nombre de cliques 26
max score among faithfull cliques of 44 forms: 930.412064935
augmenter Nombre de cliques 25
max score among faithfull cliques of 43 forms: 8

In [63]:
newParadigmes,paradigmesSwim1,characteristicsSwim1,measuresSwim1=calculerResultats(swim1ContextParadigmes,"-Swim1")
%ding

/Users/gilles/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/gilles/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/gilles/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:630: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

UG 53719 OG 3153 TP 143396 FP 819
recall 0.724463710126 precision 0.973047065849
0.830554123637


# SWIM2

## Préparation des paradigmes

In [64]:
paradigmesOriginaux=paradigmes.copy()
paradigmesSample=paradigmesOriginaux[paradigmesOriginaux["lexeme"].isin(listeTest)]

In [65]:
paradigmes=newParadigmes.copy()
paradigmesColumns=paradigmes.columns.tolist()
for c in sampleCases:
    if not c in paradigmesColumns:
        print (c)
        paradigmes[c]=np.NaN

In [66]:
cliques=[]
cliquesScores={}
cliquesListes={}

formesScores={}
formesScoresNormes={}

print (datetime.datetime.now())
%time swim2ContextParadigmes=generateCliques(contextFree=True)

2019-09-17 02:59:07.203461


escamoter Nombre de cliques 4887
max score among faithfull cliques of 48 forms: 672.224504152
anémier Nombre de cliques 7801
max score among faithfull cliques of 37 forms: 195.611686627
enquérir Nombre de cliques 9267
max score among faithfull cliques of 21 forms: 40.1378602582
réfugier Nombre de cliques 6796
max score among faithfull cliques of 40 forms: 219.91388868
assembler Nombre de cliques 7099
max score among faithfull cliques of 44 forms: 527.881916343
ensanglanter Nombre de cliques 4428
max score among faithfull cliques of 48 forms: 720.627546818
projeter Nombre de cliques 9820
max score among faithfull cliques of 43 forms: 342.76793598
conceptualiser Nombre de cliques 6092
no new faithfull clique, the previous one contained 44 forms
forger Nombre de cliques 4123
max score among faithfull cliques of 48 forms: 723.29204578
féminiser Nombre de cliques 6107
no new faithfull clique, the previous one contained 45 forms
aider Nombre de cliques 6114
no new faithfull clique, the previ

cranter Nombre de cliques 4468
max score among faithfull cliques of 48 forms: 729.164415525
revoler Nombre de cliques 6213
max score among faithfull cliques of 47 forms: 663.739156607
exceller Nombre de cliques 7449
no new faithfull clique, the previous one contained 42 forms
agrandir Nombre de cliques 6436
max score among faithfull cliques of 43 forms: 263.503277224
réveiller Nombre de cliques 9616
max score among faithfull cliques of 60 forms: 357.37337506
catalyser Nombre de cliques 6092
no new faithfull clique, the previous one contained 44 forms
relativiser Nombre de cliques 6185
no new faithfull clique, the previous one contained 40 forms
ranimer Nombre de cliques 4786
max score among faithfull cliques of 48 forms: 717.678436397
moucharder Nombre de cliques 4604
no new faithfull clique, the previous one contained 44 forms
incarner Nombre de cliques 4144
max score among faithfull cliques of 48 forms: 739.234990753
financer Nombre de cliques 5541
no new faithfull clique, the previo

max score among faithfull cliques of 43 forms: 347.87845539
repousser Nombre de cliques 5644
no new faithfull clique, the previous one contained 46 forms
affectionner Nombre de cliques 4537
max score among faithfull cliques of 48 forms: 683.208694812
incommoder Nombre de cliques 5367
no new faithfull clique, the previous one contained 44 forms
brocarder Nombre de cliques 4604
no new faithfull clique, the previous one contained 44 forms
insinuer Nombre de cliques 2714
max score among faithfull cliques of 41 forms: 335.518304102
réopérer Nombre de cliques 7785
max score among faithfull cliques of 48 forms: 638.520466383
formaliser Nombre de cliques 6092
no new faithfull clique, the previous one contained 44 forms
exploser Nombre de cliques 8448
max score among faithfull cliques of 42 forms: 441.154418409
parafer Nombre de cliques 4500
max score among faithfull cliques of 45 forms: 643.651350616
réactualiser Nombre de cliques 6092
no new faithfull clique, the previous one contained 44 for

max score among faithfull cliques of 43 forms: 247.556526828
polir Nombre de cliques 8343
max score among faithfull cliques of 42 forms: 259.568763901
proscrire Nombre de cliques 7412
max score among faithfull cliques of 41 forms: 146.048782246
prostituer Nombre de cliques 2676
max score among faithfull cliques of 41 forms: 328.359801102
cotiser Nombre de cliques 6171
no new faithfull clique, the previous one contained 41 forms
blettir Nombre de cliques 8301
max score among faithfull cliques of 43 forms: 247.022403377
biaiser Nombre de cliques 7297
no new faithfull clique, the previous one contained 43 forms
truffer Nombre de cliques 4340
max score among faithfull cliques of 48 forms: 732.797000412
destiner Nombre de cliques 4489
max score among faithfull cliques of 48 forms: 731.832807449
mépriser Nombre de cliques 6109
no new faithfull clique, the previous one contained 46 forms
mouvoir Nombre de cliques 5429
max score among faithfull cliques of 26 forms: 61.8478243683
appendre Nombr

...

max score among faithfull cliques of 48 forms: 723.704553575
enseigner Nombre de cliques 6157
no new faithfull clique, the previous one contained 39 forms
remonter Nombre de cliques 4580
max score among faithfull cliques of 48 forms: 727.704401567
grillager Nombre de cliques 4185
max score among faithfull cliques of 46 forms: 662.195191669
bander Nombre de cliques 4897
max score among faithfull cliques of 48 forms: 722.453860365
éployer Nombre de cliques 8524
max score among faithfull cliques of 40 forms: 246.48300617
modeler Nombre de cliques 10939
max score among faithfull cliques of 43 forms: 343.171221429
parfumer Nombre de cliques 4290
max score among faithfull cliques of 48 forms: 725.260761634
roder Nombre de cliques 5210
max score among faithfull cliques of 47 forms: 656.708571443
taller Nombre de cliques 6118
no new faithfull clique, the previous one contained 42 forms
mordiller Nombre de cliques 5867
no new faithfull clique, the previous one contained 39 forms
cotonner Nombre

max score among faithfull cliques of 48 forms: 717.678436397
désintégrer Nombre de cliques 4900
max score among faithfull cliques of 48 forms: 729.5990742
institutionnaliser Nombre de cliques 6092
no new faithfull clique, the previous one contained 44 forms
raffiner Nombre de cliques 4489
max score among faithfull cliques of 48 forms: 731.832807449
angoisser Nombre de cliques 5561
no new faithfull clique, the previous one contained 45 forms
accentuer Nombre de cliques 2676
max score among faithfull cliques of 41 forms: 327.929999026
infiltrer Nombre de cliques 5946
max score among faithfull cliques of 44 forms: 533.953547592
enlaidir Nombre de cliques 7223
max score among faithfull cliques of 43 forms: 264.908401519
crisser Nombre de cliques 4793
max score among faithfull cliques of 46 forms: 589.678592743
adjuger Nombre de cliques 4104
max score among faithfull cliques of 48 forms: 730.584197497
haver Nombre de cliques 6378
no new faithfull clique, the previous one contained 37 forms


max score among faithfull cliques of 48 forms: 726.965748125
cuiter Nombre de cliques 5132
max score among faithfull cliques of 48 forms: 712.551297646
prétexter Nombre de cliques 4530
max score among faithfull cliques of 48 forms: 722.073598965
javelliser Nombre de cliques 6090
no new faithfull clique, the previous one contained 44 forms
retraduire Nombre de cliques 7382
max score among faithfull cliques of 40 forms: 174.29110046
gainer Nombre de cliques 5161
no new faithfull clique, the previous one contained 43 forms
brailler Nombre de cliques 6411
max score among faithfull cliques of 40 forms: 253.519571945
urger Nombre de cliques 4104
max score among faithfull cliques of 48 forms: 726.965748125
repeindre Nombre de cliques 6934
max score among faithfull cliques of 40 forms: 260.147515871
tergiverser Nombre de cliques 5849
max score among faithfull cliques of 46 forms: 555.725850431
fondre Nombre de cliques 5005
max score among faithfull cliques of 42 forms: 212.40752706
dérouiller 

no new faithfull clique, the previous one contained 43 forms
signer Nombre de cliques 4392
max score among faithfull cliques of 42 forms: 525.201092757
pistonner Nombre de cliques 4537
max score among faithfull cliques of 48 forms: 683.208694812
étoiler Nombre de cliques 6097
no new faithfull clique, the previous one contained 42 forms
patienter Nombre de cliques 4428
max score among faithfull cliques of 48 forms: 720.627546818
évertuer Nombre de cliques 2676
max score among faithfull cliques of 41 forms: 327.929999026
minuter Nombre de cliques 4518
max score among faithfull cliques of 48 forms: 713.026607878
réédifier Nombre de cliques 7158
max score among faithfull cliques of 40 forms: 206.638175533
intellectualiser Nombre de cliques 6092
no new faithfull clique, the previous one contained 44 forms
espionner Nombre de cliques 4537
max score among faithfull cliques of 48 forms: 683.208694812
faciliter Nombre de cliques 5218
max score among faithfull cliques of 48 forms: 713.530904446


max score among faithfull cliques of 40 forms: 221.714990895
ôter Nombre de cliques 5724
max score among faithfull cliques of 42 forms: 436.717371019
folichonner Nombre de cliques 4537
max score among faithfull cliques of 48 forms: 683.208694812
CPU times: user 10h 11min 47s, sys: 1h 6min 57s, total: 11h 18min 45s
Wall time: 10h 24min 12s


In [67]:
newParadigmes,paradigmesSwim2,characteristicsSwim2,measuresSwim2=calculerResultats(swim2ContextParadigmes,"-Swim2")

UG 35389 OG 3676 TP 161726 FP 1559
recall 0.814026999003 precision 0.968645372273
0.884630847703


In [68]:
nomFichierResultats=filePrefix+"-X-Resultats.yaml"
if os.path.isfile(nomFichierResultats):
    with open(nomFichierResultats, 'r') as stream:
            resultats=yaml.load(stream)
else:
    resultats={}

if casesType:
    sampleExt=casesType
else:
    sampleExt=sampleType
sampleId=sampleNumber.strip("-")+sampleExt
resultats[sampleId]={}
resultats[sampleId]["Swim1"]={}
resultats[sampleId]["Swim1"]["UG"]=characteristicsSwim1[0]
resultats[sampleId]["Swim1"]["OG"]=characteristicsSwim1[1]
resultats[sampleId]["Swim1"]["TP"]=characteristicsSwim1[2]
resultats[sampleId]["Swim1"]["FP"]=characteristicsSwim1[3]
resultats[sampleId]["Swim1"]["Precision"]=measuresSwim1[0]
resultats[sampleId]["Swim1"]["Recall"]=measuresSwim1[1]
resultats[sampleId]["Swim1"]["F-Measure"]=measuresSwim1[2]
resultats[sampleId]["Swim2"]={}
resultats[sampleId]["Swim2"]["UG"]=characteristicsSwim2[0]
resultats[sampleId]["Swim2"]["OG"]=characteristicsSwim2[1]
resultats[sampleId]["Swim2"]["TP"]=characteristicsSwim2[2]
resultats[sampleId]["Swim2"]["FP"]=characteristicsSwim2[3]
resultats[sampleId]["Swim2"]["Precision"]=measuresSwim2[0]
resultats[sampleId]["Swim2"]["Recall"]=measuresSwim2[1]
resultats[sampleId]["Swim2"]["F-Measure"]=measuresSwim2[2]

yaml.safe_dump(resultats, file(nomFichierResultats, 'w'), encoding='utf-8', allow_unicode=True)

RepresenterError: cannot represent an object: 819

In [ ]:
%ding
%ding

In [ ]:
print ("Sample",sampleNumber.strip("-"))
print ("Swim1",characteristicsSwim1,measuresSwim1)
print ("Swim2",characteristicsSwim2,measuresSwim2)

# Fin du traitement

In [73]:
paradigmesSwim2.columns.tolist()[25:]

[u'ai2S',
 u'fi2P',
 u'fi3P',
 u'fi3S',
 u'ii3P',
 u'ii3S',
 u'is1S',
 u'is3P',
 u'is3S',
 u'pI1P',
 u'pc1S',
 u'pc2S',
 u'pc3P',
 u'pc3S',
 u'pi3S',
 u'ppFP',
 u'ppFS',
 u'ppMP',
 u'ps2P',
 u'ps3P',
 u'ps3S']

In [76]:
paradigmesSwim2.pc1S

0          abEs6rE
1        abâdOn6rE
2             None
3          abOr6rE
4          abutirE
5         absât6rE
6        absOrb6rE
7        abstjêdrE
8          abim6rE
9        aksâty6rE
10       aksEpt6rE
11            None
12       akôpaJ6rE
13        akôplirE
14        akOrd6rE
15        akrOS6rE
16            None
17         akyz6rE
18      aksElEr6rE
19        aSarn6rE
20         aSEt6rE
21            None
22        aktiv6rE
23        adapt6rE
24         admEtrE
25        admir6rE
26        adOpt6rE
27         adOr6rE
28         adOs6rE
29        adrEs6rE
           ...    
1259        EmEtrE
1260          None
1261       Enôs6rE
1262      EparJ6rE
1263       EpEl6rE
1264      Epêgl6rE
1265          None
1266       EprâdrE
1267      Epruv6rE
1268      EpHiz6rE
1269      EtablirE
1270       EtaZ6rE
1271          None
1272        EtêdrE
1273        EtâdrE
1274       Etir6rE
1275       EtOn6rE
1276       Etuf6rE
1277     Etrâgl6rE
1278       EtydirE
1279          None
1280       E

# Calcul des distances

### Préparation des fichiers de référence
La comparaison se fait sur les lexèmes de *listeTest*
- initialForms : les formes initiales
- finalForms : les formes initiales plus les formes prédites
- goldForms : les formes de référence


#### Formes initiales

#### Formes avec prédictions

#### Formes de référence
- recoder les représentations phonologiques en fonction de l'échantillon

### Soustraction des formes initiales

### Évaluation des formes

#### Problèmes de génération
- sous-génération : formes de références sans prédiction
- sur-génération : formes prédites absentes de la référence

#### Formes comparables
- *predictedForms* => formes prédites présentes dans la référence
- *actualForms* => formes de référence présentes dans les prédictions

#### Préparation de la comparaison
- *compareForms* => tableau avec les formes à comparer

#### Comparaison des formes
- *sameForms* => formes identiques dans les prédictions et la référence
- *diffForms* => formes différentes dans les prédictions et la référence

#### Décompte du nombre de formes
À cause du traitement de la surabondance, le décompte des lignes ne tient pas compte du nombre de formes. Il faut compter les formes dans chaque ligne pour obtenir le nombre total.

#### Évaluation des différences
Certaines différences ne sont qu'apparentes à cause du traitement de la surabondance sous forme de chaine concaténée. Il faut transformer les surabondances en set() pour pouvoir faire une comparaison pertinente.

#### Calcul des grandeurs de mesure

### Sauvegarde des résultats pour analyse ultérieure

## Analyse des sur-générations

### Analyse par cases

Le facteur principal de **surgénération** est la production de **participes passés** (ppMP 662, ppFS 661, ppFP 661) pour les verbes qui n'ont qu'une seule forme de participe passé invariable (ppMS) : **54.78%** (1984/3622).

### Analyse par verbes

Le facteur principal de **surgénération** tient aux **verbes vestiges** et aux **météorologiques**.

## Analyse des sous-générations

### Analyse par cases

Le facteur principal de **sous-génération** est lié à la fréquence des TAM et de certaines personnes :
- TAM : 
 - imparfait du subjonctif 45.98% (7875/17126)
 - passé simple 12.70% (2176/17126)
- Personnes :
 - 2PL 46.11% (7897/17126)
 - 1PL 32.98% (5649/17126)


### Analyse par verbes

# Fin de l'analyse des résultats

### Test de la sauvegarde de l'analyse brute

## with open(analysisPrefix+"-analyseParadigmes.pkl","rb") as input:
    savedAnalyseParadigmes = pickle.load(input)    